In [4]:
import tkinter as tk
import OpenOPC
import pywintypes
import time
pywintypes.datetime = pywintypes.TimeType


In [5]:
class NutRunner(tk.Tk):
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self,  *args, **kwargs)  
        self.parent = None
        self.opc = OpenOPC.client()
        self.opc.connect('Matrikon.OPC.Simulation.1')
        self.opc.write(('output.qReady', False))
        self.opc.write(('output.qOK', False))
        self.opc.write(('output.qNOK', False))
        self.opc.write(('output.qDone', False))
        self.opc.write(('output.qCycleRetry', False))
        self.opc.write(('output.qCycleOKPulse', False))
        self.opc.write(('output.qResults', 0))
        self.opc.write(('output.qStatus', 0))
        self.opc.write(('output.qCycleTime', False))
        self.opc.write(('output.', False))

        self.cycleNumber = 1
        self.geometry('1500x800')
        self.buttonState = 0 
        self.iInPos = tk.IntVar()
        self.iStaSeld = tk.IntVar()
        self.powerBotttonState = 0
        self.enableButtonState = 0
        self.isRunning = 0
        self.complexity = tk.StringVar()
        self.displayFrame = tk.Frame(self)
        self.displayFrame['height'] = 600
        self.displayFrame['width'] = 300
        self.displayFrame.pack()
        self.displayFrame.place(x=1100, y=100)
        self.displayFrame['bg'] = 'yellow'
        self.canvas = tk.Canvas(self.displayFrame)
        self.LEDs = dict()
        self.LEDs['qOK'] = (150, 10, 220, 80)
        self.LEDs['qReady'] = (150, 120, 220, 190)
        self.LEDs['qCycleOKPulse'] = (150, 240, 220, 310) 
        self.LEDs['qDone'] = (150, 360, 220, 430)
        self.statusPosition = (180, 500)
        self.canvas.pack()
        self.canvas['width'] = 300
        self.canvas['height'] = 600
        self.canvas.create_oval(self.LEDs['qOK'], outline="#000000", fill="#f54e42", width=2)
        self.canvas.create_text(185, 90, font="Times 12 bold", text="qOK")
        self.canvas.create_oval(self.LEDs['qReady'], outline="#000000", fill="#f54e42", width=2)
        self.canvas.create_text(183, 200, font="Times 12 bold", text="qReady")
        self.canvas.create_oval(self.LEDs['qCycleOKPulse'], outline="#000000", fill="#f54e42", width=2)
        self.canvas.create_text(180, 320, font="Times 12 bold", text="qCycleOKPluse")
        self.canvas.create_oval(self.LEDs['qDone'], outline="#000000", fill="#f54e42", width=2)
        self.canvas.create_text(184, 440, font="Times 12 bold", text="qDone")
        self.canvas.create_rectangle(140, 480, 230, 530, outline="#000000", fill="#ffffff" )
        self.canvas.create_text(self.statusPosition, font="Times 12 bold", text="Status:  0")
       
        
        # iPowerOn Button
        self.iPowerOnButton = tk.Button(self, text='POWER ON', fg='blue', bg='red', command=self.powerButtonPressed, height=4, width=20)
        self.iPowerOnButton.pack()
        self.iPowerOnButton.place(x=120, y=200)
        
        # iResetFault Button
        self.iResetFaultButton = tk.Button(self, text='RESET FAULT', fg='blue', bg='red', command=self.buttonPressed, height=4, width=20)
        self.iResetFaultButton.pack()
        self.iResetFaultButton.place(x=320, y=200)
        self.iResetFaultButton['state'] = 'disabled'
        
        # iEnable Button
        self.iEnableButton = tk.Button(self, text='ENABLE TOOL', fg='blue', bg='red', command=self.enableButtonPressed, height=4, width=20)
        self.iEnableButton.pack()
        self.iEnableButton.place(x=520, y=200)
        self.iEnableButton['state'] = 'disabled'
        
        # iStart Button
        self.iStartButton = tk.Button(self, text='START SEQUENCE', fg='blue', bg='red', command=self.startButtonPressed, height=4, width=20)
        self.iStartButton.pack()
        self.iStartButton.place(x=720, y=200)
        self.iStartButton['state'] = 'disabled'
        
        
        # iResultReset Button
        self.iResultResetButton = tk.Button(self, text='RESET TASK RESULTS', fg='blue', bg='red', command=self.resetResultButtonPressed, height=4, width=20)
        self.iResultResetButton.pack()
        self.iResultResetButton.place(x=920, y=200)
        self.iResultResetButton['state'] = 'disabled'
        
        # iInPosition Button
        self.iInPositionButton = tk.Checkbutton(self, text='In Final Position', padx=20, variable=self.iInPos, onvalue=1, offvalue=0)
        self.iInPositionButton.place(x=120, y = 350)
        self.iInPositionButton.config(font=("Times", 12, 'bold'))
         
        
        # iStaSeld Button
        self.iStaSeldButton = tk.Radiobutton(self, text='Station Selected', padx=20, variable=self.iStaSeld, value=0)
        self.iStaSeldButton.place(x=320, y = 350)
        self.iStaSeldButton.config(font=("Times", 12, 'bold'))
        
        # IO Start Address
        self.iAddrLabel = tk.Label(self, text="IO Start Address", font=("Times", 12, 'bold'))
        self.iAddrLabel.pack()
        self.iAddrLabel.place(x=120, y=450)
        self.iAddr = tk.Entry(self, font=("Times", 12, 'bold'), width = "5")
        self.iAddr.pack()
        self.iAddr.place(x=270, y=450)
        
        # OMS Device Number
        self.iDeviceNumberLabel = tk.Label(self, text="OMS Device Number", font=("Times", 12, 'bold'))
        self.iDeviceNumberLabel.pack()
        self.iDeviceNumberLabel.place(x=420, y=450)
        self.iDeviceNumber = tk.Entry(self, font=("Times", 12, 'bold'), width = "5")
        self.iDeviceNumber.pack()
        self.iDeviceNumber.place(x=620, y=450)

        # OMS Complexity (Mode)
        self.iComplexityLabel = tk.Label(self, text="OMS Complexity (Mode)", font=("Times", 12, 'bold'))
        self.iComplexityLabel.pack()
        self.iComplexityLabel.place(x=720, y=450)
        self.iComplexity = tk.Entry(self, font=("Times", 12, 'bold'), width = "5", textvariable=self.complexity)
        self.complexity.set('1')
        self.iComplexity.pack()
        self.iComplexity.place(x=920, y=450)
        
        # iInhibit
        self.iInhibitLabel = tk.Label(self, text="iInhibit", font=("Times", 12, 'bold'))
        self.iInhibitLabel.pack()
        self.iInhibitLabel.place(x=120, y=550)
        self.iInhibit = tk.Entry(self, font=("Times", 12, 'bold'), width = "5")
        self.iInhibit.pack()
        self.iInhibit.place(x=270, y=550)
        
        # iSerialID
        self.iSerialIDLabel = tk.Label(self, text="iSerialID", font=("Times", 12, 'bold'))
        self.iSerialIDLabel.pack()
        self.iSerialIDLabel.place(x=420, y=550)
        self.iSerialID = tk.Entry(self, font=("Times", 12, 'bold'), width = "5")
        self.iSerialID.pack()
        self.iSerialID.place(x=620, y=550)
        
        # iHMI_String
        self.iHMI_StringLabel = tk.Label(self, text="HMI System Description", font=("Times", 12, 'bold'))
        self.iHMI_StringLabel.pack()
        self.iHMI_StringLabel.place(x=720, y=550)
        self.iHMI_String = tk.Entry(self, font=("Times", 12, 'bold'), width = "5")
        self.iHMI_String.pack()
        self.iHMI_String.place(x=920, y=550)
        
        # iStartFaultTime
        self.iStartFaultTimeLabel = tk.Label(self, text="Timer Preset", font=("Times", 12, 'bold'))
        self.iStartFaultTimeLabel.pack()
        self.iStartFaultTimeLabel.place(x=120, y=650)
        self.iStartFaultTime = tk.Entry(self, font=("Times", 12, 'bold'), width = "5")
        self.iStartFaultTime.pack()
        self.iStartFaultTime.place(x=270, y=650)
        
        # iCycleFaultTime
        self.iCycleFaultTimeLabel = tk.Label(self, text="Cycle Timeout Preset", font=("Times", 12, 'bold'))
        self.iCycleFaultTimeLabel.pack()
        self.iCycleFaultTimeLabel.place(x=420, y=650)
        self.iCycleFaultTime = tk.Entry(self, font=("Times", 12, 'bold'), width = "5")
        self.iCycleFaultTime.pack()
        self.iCycleFaultTime.place(x=620, y=650)
    
    def writeStatus(self):
        self.canvas.create_rectangle(140, 480, 230, 530, outline="#000000", fill="#ffffff" )
        self.canvas.create_text(self.statusPosition, font="Times 12 bold", text="Status:  " + str(self.cycleNumber))
    def setLED(self, led):
        self.canvas.create_oval(self.LEDs[led], outline="#000000", fill="#81eb8d", width=2)
    def resetLED(self, led):
        self.canvas.create_oval(self.LEDs[led], outline="#000000", fill="#f54e42", width=2)

    def powerButtonPressed(self):
        if self.powerBotttonState == 0:
            self.powerBotttonState = 1
            self.iPowerOnButton['bg'] = 'green'
            self.iEnableButton['state'] = 'normal'
            self.iPowerOnButton['text'] = 'POWER OFF'
        else:
            self.powerBotttonState = 0
            self.iResetFaultButton['state'] = 'disabled'
            self.iEnableButton['state'] = 'disabled'
            self.iStartButton['state'] = 'disabled'
            self.iResultResetButton['state'] = 'disabled'
            self.iEnableButton['bg'] = 'red'
            self.iEnableButton['text'] = 'ENABLE TOOL'
            self.iPowerOnButton['bg'] = 'red'
            self.iPowerOnButton['text'] = 'POWER ON'
            
    def enableButtonPressed(self):
        if self.enableButtonState == 0:
            self.opc.write(('output.qReady', True))
            self.setLED('qReady')
            self.enableButtonState = 1
            self.iResetFaultButton['state'] = 'normal'
            self.iStartButton['state'] = 'normal'
            self.iResultResetButton['state'] = 'normal'
            self.iEnableButton['bg'] = 'green'
            self.iEnableButton['text'] = 'DISABLE TOOL'
            
        else:
            self.opc.write(('output.qReady', False))
            self.resetLED('qReady')
            self.enableButtonState = 0
            self.iResetFaultButton['state'] = 'disabled'
            self.iStartButton['state'] = 'disabled'
            self.iResultResetButton['state'] = 'disabled'
            self.iEnableButton['bg'] = 'red'
            self.iEnableButton['text'] = 'ENABLE TOOL'
            
    def setQCycleOKPulse(self):
        self.opc.write(('output.qCycleOKPulse', True))
        self.setLED('qCycleOKPulse')
    
    def resetQCycleOKPulse(self):
        self.opc.write(('output.qCycleOKPulse', False))
        self.resetLED('qCycleOKPulse')
        
    def setCycleTerminationValues(self):
        self.opc.write(('output.qCycleTime', 3000)) 
        self.opc.write(('output.qStatus', self.cycleNumber))
        self.writeStatus()
        if self.cycleNumber == int(self.complexity.get()):
            self.opc.write(('output.qDone', True))
            self.setLED('qDone')
            self.opc.write(('output.qResult', int(self.complexity.get())))
            self.opc.write(('output.qOK', False))
            self.resetLED('qOK')
            self.opc.write(('output.qReady', True))
            self.setLED('qReady')
            self.cycleNumber = 0
        else:
            self.cycleNumber += 1
    def startButtonPressed(self):
        if self.isRunning == 0:
            self.isRunning=1
            self.opc.write(('output.qReady', False))
            self.resetLED('qReady')
            self.setLED('qOK')
            self.opc.write(('output.qOK', True))
            self.iStartButton['state'] = 'normal'
            n = int(self.complexity.get())
            self.cycleNumber = 0
            self.opc.write(('output.qStatus', 0))
            self.writeStatus()
            self.cycleNumber = 1
            
            for i in range(n):
                # the cycle happens for 3 seconds
                self.after(6000*i + 0,lambda : self.setQCycleOKPulse()) 
                self.after(6000*i + 1000,lambda : self.resetQCycleOKPulse())
                # after a 2 second cycle OK pulse all the cycle 
                # termination signals are appropriately set
                self.after(6000*i + 3000,lambda : self.setCycleTerminationValues())
            self.isRunning = 0            
            
        else:
            self.opc.write(('output.qNOK', True))
            while self.isRunning == 1:
                pass
            self.opc.write(('output.qCycleRetryPulse', True))
            time.sleep(2)
            self.opc.write(('output.qCycleRetryPulse', False)) 
               
    def resetResultButtonPressed(self):
        self.isRunning = 0
        self.opc.write(('output.qCycleOKPulse', False)) 
        self.resetLED('qCycleOKPulse')
        self.opc.write(('output.qCycleRetryPulse', False)) 
        self.opc.write(('output.qStatus', 0))
        self.cycleNumber = 0
        self.writeStatus()
        self.opc.write(('output.qDone', False)) 
        self.resetLED('qDone')
        self.opc.write(('output.qCycleTime', 0))
        self.opc.write(('output.qResult', 0))
    def buttonPressed(self):
        self.buttonState = 1
    def run(self):
        self.mainloop()
        
    def __del__(self): 
        self.opc.close()
       

In [6]:
N = NutRunner()
N.run()